# Slowed and Reverb Music Video Maker

References to project:
- [Youtube video downloader on SO](https://stackoverflow.com/questions/65802599/how-to-download-youtube-video-in-the-highest-quality-available)
- [File Chooser widget](https://pypi.org/project/ipyfilechooser/)
- [Attach GIF to video with Moviepy](https://github.com/Zulko/moviepy/issues/127)

- References for download progress bar from: [1](https://stackoverflow.com/questions/56197879/how-to-use-progress-bar-in-pytube) and [2](https://stackoverflow.com/questions/56197879/how-to-use-progress-bar-in-pytube).

- (Attention) MoviePy too slow to make video. Potential fixes [here](https://stackoverflow.com/questions/56413813/concat-videos-too-slow-using-python-moviepy)

pip installs:
- pytube
- ipyfilechooser
- moviepy

Need to install ImageMagick [here](https://imagemagick.org/script/download.php) for TextClip function to work. During installation, check the following two options:
- Associate supported file extensions with ImageMagick
- Install legacy utilities (e.g. convert)

Moviepy code block does not work for every computer? Check `pip list`

Import relevant libraries for this project.

In [1]:
#YouTube library
import pytube
from pytube.cli import on_progress
import sys
from ipyfilechooser import FileChooser
import ipywidgets as widgets

### Specify the PATH for the videos to be saved here:

Already have a .mp4 file saved on computer ready? Skip the following steps and head to section `Select a GIF to loop for the entire duration of the video`.

In [2]:
PATH = input("> ")

> C:\Users\asher\OneDrive\personal projects\snr maker


### Insert the URL of the song you want to convert. 

In [6]:
#video downloader
def download():
    video_url = url
    try:
        youtube = pytube.YouTube(video_url, on_progress_callback = progress_func)
        video = youtube.streams.filter(file_extension='mp4', only_audio=False, progressive=True).last()
        video.download(PATH)
        print("Download Complete!   ")
    except Exception as e:
        print(e)
        print("Video could not be downloaded.")

#progress bar function        
def progress_func(stream, chunk, bytes_remaining):
    current = ((stream.filesize - bytes_remaining)/stream.filesize)
    percent = ('{0:.1f}').format(current * 100)
    progress = int(50 * current)
    status = '█' * progress + '-' * (50 - progress)
    sys.stdout.write(' ↳ |{bar}| {percent}%\r'.format(bar = status, percent = percent))
    sys.stdout.flush()
    
url = input("> ")
download()

> https://youtu.be/6Tldvp8adQ4
Download Complete!   


### Select a GIF to loop for the entire duration of the video.

In [3]:
fc = FileChooser()
# Can put PATH into FileChooser argument if GIFs are saved in the same directory as the .mp4 files
display(fc)

FileChooser(path='C:\Users\asher\OneDrive\personal projects\snr maker', filename='', title='', show_hidden=Fal…

In [4]:
print('You have selected: \n')
print(fc.selected_filename)
selected_gif_path = fc.selected

file = open(selected_gif_path, "rb")
image = file.read()
widgets.Image(
    value=image,
    format='gif',
    width=300,
    height=400,
)

You have selected: 

cowboy-bebop-ed.gif


Image(value=b'GIF89a\xdc\x00\xa5\x00\xf7\xc0\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x…

## Choose a video to be combined with the GIF

In [5]:
vc = FileChooser()
# Can put PATH into FileChooser argument if vids are saved in the same directory
display(vc)

FileChooser(path='C:\Users\asher\OneDrive\personal projects\snr maker', filename='', title='', show_hidden=Fal…

In [9]:
#Customize Font in next cell block? See the list of available options by uncomment the next line:
#TextClip.list('font')

['AdobeDevanagari-Bold',
 'AdobeDevanagari-BoldItalic',
 'AdobeDevanagari-Italic',
 'AdobeDevanagari-Regular',
 'Agency-FB',
 'Agency-FB-Bold',
 'Algerian',
 'Arial',
 'Arial-Black',
 'Arial-Bold',
 'Arial-Bold-Italic',
 'Arial-Italic',
 'Arial-Narrow',
 'Arial-Narrow-Bold',
 'Arial-Narrow-Bold-Italic',
 'Arial-Narrow-Italic',
 'Arial-Rounded-MT-Bold',
 'Bahnschrift',
 'Baskerville-Old-Face',
 'Bauhaus-93',
 'Bell-MT',
 'Bell-MT-Bold',
 'Bell-MT-Italic',
 'Berlin-Sans-FB',
 'Berlin-Sans-FB-Bold',
 'Berlin-Sans-FB-Demi-Bold',
 'Bernard-MT-Condensed',
 'Blackadder-ITC',
 'Bodoni-MT',
 'Bodoni-MT-Black',
 'Bodoni-MT-Black-Italic',
 'Bodoni-MT-Bold',
 'Bodoni-MT-Bold-Italic',
 'Bodoni-MT-Condensed',
 'Bodoni-MT-Condensed-Bold',
 'Bodoni-MT-Condensed-Bold-Italic',
 'Bodoni-MT-Condensed-Italic',
 'Bodoni-MT-Italic',
 'Bodoni-MT-Poster-Compressed',
 'Book-Antiqua',
 'Book-Antiqua-Bold',
 'Book-Antiqua-Bold-Italic',
 'Book-Antiqua-Italic',
 'Bookman-Old-Style',
 'Bookman-Old-Style-Bold',
 'Boo

In [10]:
import moviepy.editor as mp
from moviepy.editor import ColorClip,TextClip
import os #for video file name

#Selected Video
PATH_V = vc.selected
video_dl = mp.VideoFileClip(PATH_V)

#Dark background
black_clip = ColorClip(size = video_dl.size, color = [0,0,0]).set_duration(video_dl.duration)

#Selected GIF
animated_gif = (mp.VideoFileClip(selected_gif_path)
          .loop()
          .set_duration(video_dl.duration)
          .resize(width=200, height=300)
          .margin(right=8, top=8, opacity=0)
          .set_pos(("center","center")))

#video file rename
vidfilename = vc.selected_filename
vidfilename_abbrv = os.path.splitext(vidfilename)[0]
#Title
name_caption = vidfilename_abbrv + ' (slowed)'
title_clip = TextClip(txt=name_caption, fontsize=30, font= 'Brush-Script-MT-Italic', color='white')
title = title_clip.set_pos('bottom').set_duration(video_dl.duration)

final = mp.CompositeVideoClip([video_dl, black_clip, animated_gif, title])
final.write_videofile("test.mp4")

Moviepy - Building video test.mp4.
MoviePy - Writing audio in testTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video test.mp4



Moviepy - Done !
Moviepy - video ready test.mp4
